# MMCV 세팅

In [1]:
import torch
print(torch.__version__)

1.12.1+cu113


In [2]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.1/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.1/index.html
     |████████████████████████████████| 575 kB 30.4 MB/s 
     |████████████████████████████████| 190 kB 79.4 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.6.2-cp37-cp37m-linux_x86_64.whl size=24341524 sha256=d0efb5027dddaee254ac9dea67045f6e5bcb2e13a8d30bb0d73b57930d5e3a5a
  Stored in directory: /root/.cache/pip/wheels/51/0a/2c/990e1866b4c28e9a70ae3a0a304eeaf4711e85a87c647b8b79
Successfully built mmcv-full


In [3]:
!cd /content/drive/MyDrive/mmdetection; python setup.py install

running install
running bdist_egg
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mmdet
copying build/lib/mmdet/version.py -> build/bdist.linux-x86_64/egg/mmdet
copying build/lib/mmdet/__init__.py -> build/bdist.linux-x86_64/egg/mmdet
creating build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib/mmdet/utils/split_batch.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib/mmdet/utils/replace_cfg_vals.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib/mmdet/utils/misc.py -> build/bdist.linux-x86_64/egg

# Mask RCNN

In [1]:
# 런타임 다시 시작 후 아래 수행
import os
import mmcv
from mmdet.apis import init_detector, inference_detector

In [2]:
!wget -O /content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r101_fpn_2x_coco/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth

--2022-10-18 14:08:04--  https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r101_fpn_2x_coco/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.74.197.92
Connecting to download.openmmlab.com (download.openmmlab.com)|47.74.197.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254094589 (242M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth’

/content/drive/MyDr 100%[===================>] 242.32M  10.6MB/s    in 23s     

2022-10-18 14:08:27 (10.6 MB/s) - ‘/content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth’ saved [254094589/254094589]



In [3]:
!ls -lia /content/drive/MyDrive/mmdetection/checkpoints

total 496274
7440 -rw------- 1 root root 254089599 Oct  9 14:03 mask_rcnn_r101_fpn_1x_coco_20200204-1efe0ed5.pth
7436 -rw------- 1 root root 254094589 Nov  2  2021 mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth


In [4]:
config_file = '/content/drive/MyDrive/mmdetection/configs/mask_rcnn/mask_rcnn_r101_fpn_2x_coco.py'
checkpoint_file = '/content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth'

In [5]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset

@DATASETS.register_module(force=True)
class WATERDataset(CocoDataset):
  CLASSES = ('water', 'stone') 

In [6]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoi

In [7]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'WATERDataset'
cfg.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'

cfg.data.train.type = 'WATERDataset'
cfg.data.train.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.train.ann_file = 'train.json'
cfg.data.train.img_prefix = ''

cfg.data.val.type = 'WATERDataset'
cfg.data.val.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.val.ann_file = 'val.json'
cfg.data.val.img_prefix = ''

cfg.data.test.type = 'WATERDataset'
cfg.data.test.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = ''

cfg.model.roi_head.bbox_head.num_classes = 2
cfg.model.roi_head.mask_head.num_classes = 2

cfg.load_from = '/content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth'

cfg.work_dir = '/content/drive/MyDrive/epoch24/log_save'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

cfg.evaluation.metric = ['bbox', 'segm']
cfg.evaluation.interval = 12
cfg.checkpoint_config.interval = 12

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.device='cuda'

In [8]:
print(cfg.pretty_text)

model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoi

In [9]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성. 
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.94s)
creating index...
index created!


In [10]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('water', 'stone')


In [11]:
import os.path as osp

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-10-18 14:08:42,555 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2022-10-18 14:08:43,303 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth


Done (t=0.74s)
creating index...
index created!


2022-10-18 14:08:43,645 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([3, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([3]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([8, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([8]).
size mismatch for roi_head.mask_head.conv_logits.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for roi_head.mask_head.conv_logits.bias

...

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-10-18 18:14:39,512 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.407
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.597
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.463
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.702
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.029
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.03s).
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*


/usr/local/lib/python3.7/dist-packages/pycocotools/cocoeval.py:378: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)
2022-10-18 18:14:39,735 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.602
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.499
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large

DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.04s).


## 테스트 데이터 세트에 Evaluation 및 Inference 적용하기

In [12]:
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [13]:
cfg.data.samples_per_gpu = 1
cfg.dump('/content/drive/MyDrive/epoch24/log_save/mask_rcnn_conf.py')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

next(iter(data_loader))

loading annotations into memory...
Done (t=0.81s)
creating index...
index created!


{'img_metas': [DataContainer([[{'filename': '/content/drive/MyDrive/water detection/WATER_Dataset_coco/JPEGImages/00004785.jpg', 'ori_filename': 'JPEGImages/00004785.jpg', 'ori_shape': (720, 1280, 3), 'img_shape': (750, 1333, 3), 'pad_shape': (768, 1344, 3), 'scale_factor': array([1.0414063, 1.0416666, 1.0414063, 1.0416666], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]])],
 'img': [tensor([[[[-0.5253, -0.5253, -0.5253,  ...,  0.0000,  0.0000,  0.0000],
            [-0.5424, -0.5424, -0.5253,  ...,  0.0000,  0.0000,  0.0000],
            [-0.5424, -0.5424, -0.5424,  ...,  0.0000,  0.0000,  0.0000],
            ...,
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000

In [14]:
checkpoint_file = '/content/drive/MyDrive/epoch24/log_save/epoch_24.pth' 
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')
model_ckpt = MMDataParallel(model_ckpt, device_ids=[0])
outputs = single_gpu_test(model_ckpt, data_loader, True, '/content/drive/MyDrive/epoch24/show_test_output', 0.3)
metric = dataset.evaluate(outputs, metric=['bbox', 'segm'])
print(metric)

load checkpoint from local path: /content/drive/MyDrive/epoch24/log_save/epoch_24.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 153/153, 2.3 task/s, elapsed: 65s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.12s).
Accumulating evaluation results...
DONE (t=0.03s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.506
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.388
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.564
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:

/usr/local/lib/python3.7/dist-packages/mmdet-2.25.2-py3.7.egg/mmdet/datasets/coco.py:474: UserWarning: The key "bbox" is deleted for more accurate mask AP of small/medium/large instances since v2.12.0. This does not change the overall mAP calculation.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/pycocotools/cocoeval.py:378: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)


DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.03s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.366
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.502
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.027
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.653
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.160
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.301
 Average Recall     (AR) @[ IoU=0.50:0.95 | 

## Video Inference

In [1]:
# RAM을 다 사용해서 런타임 다시 실행됨

import mmcv
import cv2
from mmdet.apis import init_detector, inference_detector
from mmcv import Config

In [2]:
config_file = '/content/drive/MyDrive/mmdetection/configs/mask_rcnn/mask_rcnn_r101_fpn_2x_coco.py'
cfg = Config.fromfile(config_file)

In [3]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'WATERDataset'
cfg.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'

cfg.data.train.type = 'WATERDataset'
cfg.data.train.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.train.ann_file = 'train.json'
cfg.data.train.img_prefix = ''

cfg.data.val.type = 'WATERDataset'
cfg.data.val.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.val.ann_file = 'val.json'
cfg.data.val.img_prefix = ''

cfg.data.test.type = 'WATERDataset'
cfg.data.test.data_root = '/content/drive/MyDrive/water detection/WATER_Dataset_coco/'
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = ''

cfg.model.roi_head.bbox_head.num_classes = 2
cfg.model.roi_head.mask_head.num_classes = 2

cfg.load_from = '/content/drive/MyDrive/mmdetection/checkpoints/mask_rcnn_r101_fpn_2x_coco_bbox_mAP-0.408__segm_mAP-0.366_20200505_071027-14b391c7.pth'

cfg.work_dir = '/content/drive/MyDrive/epoch24/log_save'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

cfg.evaluation.metric = ['bbox', 'segm']
cfg.evaluation.interval = 12
cfg.checkpoint_config.interval = 12

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.device='cuda'

In [4]:
checkpoint_file = '/content/drive/MyDrive/epoch24/log_save/epoch_24.pth'
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/drive/MyDrive/epoch24/log_save/epoch_24.pth


In [5]:
video_reader = mmcv.VideoReader('/content/drive/MyDrive/water detection/WATER_Dataset_coco/water_test_2.mp4')
video_writer = None
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('/content/drive/MyDrive/water detection/WATER_Dataset_coco/water_test_2_output.mp4', fourcc, video_reader.fps,(video_reader.width, video_reader.height))

for frame in mmcv.track_iter_progress(video_reader):
  result = inference_detector(model_ckpt, frame)
  frame = model_ckpt.show_result(frame, result, score_thr=0.5)

  video_writer.write(frame)

if video_writer:
        video_writer.release()

[                                                  ] 0/480, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/mmdet-2.25.2-py3.7.egg/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 480/480, 2.7 task/s, elapsed: 178s, ETA:     0s
